# API DEMO

Now that you have some general knowledge of APIs and how to call them with the `requests` library, let's walk you through a concrete example of how to navigate an API's documentation in order to start querying it and get the information that you are looking for!

## What you are going to learn in this course 🧐🧐

This course will give you a step by step demo of how you should generally approach the discovery and usage of a new API. Here's the outline:

* Explore an API's documentation
* Start querying the RATP API

## Explore an API's documentation

Creating an API is one thing, you are making an interface available to your users so they can get information through programs. However, apart from those who created the API, no one would know how to use it. This is why every API comes with a documentation to help users find what they are looking for!

Let's take a look at the documentation for RATP's API (the unofficial french public transport network's API) available at the following [link](https://api-ratp.pierre-grimaud.fr/v4/).

The homepage looks like this:

![ratp_api_homepage](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/0-M04-data-collection/ratp_api_homepage.PNG)

The homepage actually contains the api's documentation! In this case, and it's actually pretty common, all the url we will request for this api will start by `https://api-ratp.pierre-grimaud.fr/v4/` which is the homepage url, then we can add suffixes to this url to get various information.

The various urls you can access to get data from an API are called **endpoints**! This a word that will appear in almost all the API documentations you will run into, so remember it means that it is the url you may request to get a response from the API.

### API call with generic url/endpoint

As you can see in the documentation, some of the urls you can request are generic, and others must include specific information like `type` or `code`.

Let's focus first on the this url `https://api-ratp.pierre-grimaud.fr/v4/lines`, click this section of the documentation to get additional information about this endpoint!

![ratp_lines](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/0-M04-data-collection/ratp_lines.PNG)

There are several we learn from reading this documentation:

* **Purpose**: we now know that this endpoint should give us informations about all the lines from the ratp network
* **Parameters**: there are none
* **Responses**: the only response type documented is 200 and the response data type is json!

With this in mind let's try it!

In [1]:
import requests

r =  requests.get("https://api-ratp.pierre-grimaud.fr/v4/lines")
print("Response code:",r,"\n \n")
print("Response data:\n")
r.json()

Response code: <Response [200]> 
 

Response data:



{'result': {'metros': [{'code': '1',
    'name': 'Métro 1',
    'directions': 'La Defense / Chateau de Vincennes',
    'id': '62'},
   {'code': '2',
    'name': 'Métro 2',
    'directions': 'Porte Dauphine / Nation',
    'id': '67'},
   {'code': '3',
    'name': 'Métro 3',
    'directions': 'Pont de Levallois - Becon / Gallieni',
    'id': '68'},
   {'code': '3b',
    'name': 'Métro 3b',
    'directions': 'Gambetta / Porte des Lilas',
    'id': '69'},
   {'code': '4',
    'name': 'Métro 4',
    'directions': 'Bagneux - Lucie Aubrac / Porte de Clignancourt',
    'id': '70'},
   {'code': '5',
    'name': 'Métro 5',
    'directions': "Place d'Italie / Bobigny - Pablo Picasso",
    'id': '71'},
   {'code': '6',
    'name': 'Métro 6',
    'directions': 'Charles de Gaulle - Etoile / Nation',
    'id': '72'},
   {'code': '7',
    'name': 'Métro 7',
    'directions': "Villejuif-L. Aragon / Mairie d'Ivry / la Courneuve - 8 Mai 1945",
    'id': '73'},
   {'code': '7b',
    'name': 'Métro 7b',
  

The request was successful! 

The data lists all the different transport lines managed by ratp. It lists various pieces of information about the line:

* **code**: a number associated with the line
* **name**: the commonly used name of the line
* **directions**: the final destinations of the line
* **id**: an identification number for the line

### API call with specific endpoint/url

Now that we have covered the easy topic of calling generic API endpoints let's explore endpoints that contain options.

For example, the `https://api-ratp.pierre-grimaud.fr/v4/lines/{type}/{code}` accepts two options, `type`, and `code`. Let's take a look at what the documentation says:

![ratp_lines_option](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/0-M04-data-collection/ratp_lines_options.PNG)

The documentation tells you this endpoint will give you information about a specific ratp line. It also decribes what is expected for the two options:

* `type`: is one of the following options `metros`, `rers`, `tramways`, `buses`, `noctiliens`, and determines the subsection of lines you are trying to get information on
* `code`: indicates the code of the line, it is the data associated with `code` key in the previous json file obtained from the API

Two response codes may be recieved from this endpoint, either 200 for success or 400 in case the `type` or `code` option you asked for does not exist.

Let's give it a try!

In [2]:
r = requests.get("https://api-ratp.pierre-grimaud.fr/v4/lines/metros/1")

print("Response code:", r, "\n \n")
print("Response data:\n")
r.json()

Response code: <Response [200]> 
 

Response data:



{'result': {'code': '1',
  'name': 'Métro 1',
  'directions': 'La Defense / Chateau de Vincennes',
  'id': '62'},
 '_metadata': {'call': 'GET /lines/metros/1',
  'date': '2022-01-12T14:38:05+01:00',
  'version': 4}}

We were able to get information about line 1 of the Parisian metro!

### API call with payload

The ratp API we just gave you a tour of is rather simple, either you know exactly which endpoint to call and you'd get the information, or you don't. Now there are some API that let you run `GET` queries in a way that is a little more flexible.

Let's take the example of the [teleport API](https://developers.teleport.org/api/getting_started/#search_name), an API that contains data on cities around the worlds regarding quality of likfe. This API comes with a feature that lets you search cities by name.

There are two ways of calling the API to search for cities, one uses an url and the other leverages the ability of the `requests` library to handle payload.

If we look at the API documentation, we notice that the example given is `https://api.teleport.org/api/cities/?search=San%20Francisco` to search for San Francisco. However this is not necessarily the best way to proceed for a number of reasons:

* If the number of options becomes higher then the url will get more complicated with very little legibility
* Typing a url directly is not really intuitive and is prone to error
* Special characters aren't allowed in urls, which makes some queries espacially difficult to write

Which is why it is often useful to use the `params` argument of the `requests.get` method. Let's give a shot demonstration.

In [3]:
response1 = requests.get("https://api.teleport.org/api/cities/?search=San%20Francisco")
response1.json()

{'_embedded': {'city:search-results': [{'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:5391959/'}},
    'matching_alternate_names': [{'name': 'San Francisco'}],
    'matching_full_name': 'San Francisco, California, United States'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:3652462/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Quito'}],
    'matching_full_name': 'Quito, Pichincha, Ecuador (San Francisco de Quito)'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:3493146/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Macorís'},
     {'name': 'San Francisco de Macoris'}],
    'matching_full_name': 'San Francisco de Macorís, Duarte, Dominican Republic'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:5454711/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Albuquerque'},
     {'name': 'San F

In [4]:
response2 = requests.get(" https://api.teleport.org/api/cities",params={"search":"San Francisco"})
response2.json()

{'_embedded': {'city:search-results': [{'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:5391959/'}},
    'matching_alternate_names': [{'name': 'San Francisco'}],
    'matching_full_name': 'San Francisco, California, United States'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:3652462/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Quito'}],
    'matching_full_name': 'Quito, Pichincha, Ecuador (San Francisco de Quito)'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:3493146/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Macorís'},
     {'name': 'San Francisco de Macoris'}],
    'matching_full_name': 'San Francisco de Macorís, Duarte, Dominican Republic'},
   {'_links': {'city:item': {'href': 'https://api.teleport.org/api/cities/geonameid:5454711/'}},
    'matching_alternate_names': [{'name': 'San Francisco de Albuquerque'},
     {'name': 'San F

In [5]:
response1.json()==response2.json()

True

Responses for both queries are identical! So remember to use payloads every time you need to add extra parameters to your API call, it is very useful and helps prevent errors!

### API call with POST method

In the case of search, or anything else using payloads to add parameters to your query, you are only trying to access data which is already here, it all pre-exists your request. In other cases, for example if you are trying to get a predictive model's prediction, then the response you will get will be built on the spot depending on the data you sent.

This is actually where the post method comes in handy, because it lets you send data to the API that will be transformed in some way to create the response. 

A really good example of such a usecase is a machine translation API, like [Google Translate](https://rapidapi.com/googlecloud/api/google-translate1/). APIs registered on **rapideapi** are really easy to use because the platform registers API and gives standardised documentation, which means you won't have to re-learn how to use the API every single time. It even lets you indicate with what language and what library you're using to access the API. So if we choose `pyhthon` `requests` we see this:

```python
import requests

url = "https://google-translate1.p.rapidapi.com/language/translate/v2"

payload = "q=Hello%2C%20world!&target=es&source=en"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'accept-encoding': "application/gzip",
    'x-rapidapi-host': "google-translate1.p.rapidapi.com",
    'x-rapidapi-key': "d448db74f1msh71600bf0b7eb394p1467b7jsn2923992aa0d0"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)
```

Let's try and use it in an even simpler way

In [7]:
url = "https://google-translate1.p.rapidapi.com/language/translate/v2"
data = {"q":"Hello, World!", "target":"es", "source":"en"}
headers = {
    'x-rapidapi-key': "d448db74f1msh71600bf0b7eb394p1467b7jsn2923992aa0d0"
    }

translation = requests.post(url, data=data, headers=headers)
translation.json()

{'data': {'translations': [{'translatedText': '¡Hola Mundo!'}]}}

The first element of the query had to be the endpoint we are trying to access. Then we had to include the data with the sentence we wish to translate, the target language and source language. Then the `headers` is used to feed our authentication key to the API so it knows we are authorized and have'nt exceeded our query quota.

## API limitations and quotas

Most APIs put in place protections such as authentication procedures, limits and quotas. There are several reasons for this:

* If too many queries hit your API at the same time, the machine on which the API is running might not be able to handle them all and crash, meaning the API will be down until it gets rebooted.
* Having your API running on a machine or a group of machines to match incoming demand is costly, therefore you want to prevent people from over using your API to prevent costs blowing through the roof.
* Let's take the example of Twitter, a huge part of its data is actually public, since it's comprised of what public  profiles post, therefore if there were no limitations on Twitter's API, people could get all of twitter's publicly available data and use it for commercial purposes. However this is already what twitter is trying to do to sell advertising for example, to leverage there data to do ad targeting, if everyone could do the same twitter would lose a huge competitive advantage.
* Some APIs are not meant to be public, companies and institutions use APIs to make specific data available to their teams but do not whish the general public to have access.

It is not generally dangerous to exceed quotas or to try and call an API with the wrong access key because most of the time you will simply get an error stating the reason for your query not succeeding. Just read carefully trough the API's documentation to understand its security standards and limitations in order to avoid unpleasant surprises such as having to wait 24 hours for your quota to be filled up again!

## Concluding remarks
By now you should have a much better understanding of how API work and how to get started with brand new APIs. Let's sum up what we have learned and list some best practices:

* ALWAYS read the documentation carefully before writing API calls
* Start with test queries that resolve quickly before attempting to get massive amounts of data
* Pay attention to athentication instructions and quotas in order to gauge the amount of calls you'd be able to make to avoid getting stuck
* Try and use the full potential of the methods `.get()`, and `.post()` from the `requests` library instead of trying to write the full query using the url.